In [1]:
import pandas as pd
import re

In [2]:
# Map to build FIPS code
state_to_FIPS = {
    'AL':'01',
    'AK':'02',
    'AZ':'04',
    'AR':'05',
    'CA':'06',
    'CO':'08',
    'CT':'09',
    'DE':'10',
    'FL':'12',
    'GA':'13',
    'HI':'15',
    'ID':'16',
    'IL':'17',
    'IN':'18',
    'IA':'19',
    'KS':'20',
    'KY':'21',
    'LA':'22',
    'ME':'23',
    'MD':'24',
    'MA':'25',
    'MI':'26',
    'MN':'27',
    'MS':'28',
    'MO':'29',
    'MT':'30',
    'NE':'31',
    'NV':'32',
    'NH':'33',
    'NJ':'34',
    'NM':'35',
    'NY':'36',
    'NC':'37',
    'ND':'38',
    'OH':'39',
    'OK':'40',
    'OR':'41',
    'PA':'42',
    'RI':'44',
    'SC':'45',
    'SD':'46',
    'TN':'47',
    'TX':'48',
    'UT':'49',
    'VT':'50',
    'VA':'51',
    'WA':'52',
    'WV':'54',
    'WI':'55',
    'WY':'56',
    'AS':'60',
    'GU':'66',
    'MP':'69',
    'PR':'72',
    'VI':'78',
}

In [3]:
def convert_ORI_to_FIPS(ORI_code):
    county_code = ORI_code[2:5]
    state = ORI_code[0:2]
    state_number = state_to_FIPS[state]
    FIPS_Code = state_number + county_code
    return FIPS_Code

In [4]:
# Read the similarity output files
with open('SIM_OUTPUT_2016-2020.txt') as f:
    dictionary = {}
    rx = re.compile("-----------------------------FBI AGENCY:")
    while True:
        line = f.readline()
        if line:
            m = rx.search(line)
            if m:
                next_line = f.readline()
                parts = next_line.split(",")
                array = parts[1].split(" ")
                target_agency = array[3].split(":")[1]
                target_county = convert_ORI_to_FIPS(target_agency)
                #target_agency = final_part
                ## Find similar matches
                l = f.readline()
                dictionary[target_county] = []
                for i in range(20):
                    try:
                        l = f.readline()
                        similar_agency = l.split(",")[0]
                        if len(similar_agency) > 3:
                            similar_agency = similar_agency[-8:-1]
                            similar_county = convert_ORI_to_FIPS(similar_agency)
                            #print(similar_county)
                            dictionary[target_county].append(similar_county)
                    except:
                        print("Parsing error")
        else:
            break

In [13]:
# Read the combined csv
# combined_df = pd.read_csv("/content/drive/MyDrive/BDA/CSE_545_Project/Arrest_Data/combined_csv_2001_2005.csv")

In [5]:
import os

combined_df = None
for i in range(1, 6):
    if i >= 10:
        name = "ASR_20" + str(i) + ".csv"
    else:
        name = "ASR_200" + str(i) + ".csv"
    full_path = os.path.join("E:\SBU_acad\Spring 22\CSE 545\Arrest_Data_new", name)
    print("Reading:", full_path)
    data = pd.read_csv(full_path)
    combined_df = pd.concat([combined_df, data])

Reading: E:\SBU_acad\Spring 22\CSE 545\Arrest_Data_new\ASR_2001.csv


c:\users\dhruv\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Reading: E:\SBU_acad\Spring 22\CSE 545\Arrest_Data_new\ASR_2002.csv
Reading: E:\SBU_acad\Spring 22\CSE 545\Arrest_Data_new\ASR_2003.csv
Reading: E:\SBU_acad\Spring 22\CSE 545\Arrest_Data_new\ASR_2004.csv
Reading: E:\SBU_acad\Spring 22\CSE 545\Arrest_Data_new\ASR_2005.csv


In [6]:
fips_mapping = {
    '48HPD': '48201',
    '48SPD': '48029',
    '17CPD': '17031',
    '42PEP': '42101',
    '13APD': '13121'
}

In [7]:
combined_df['County_code'] = combined_df.ORI_Code.str[2:5]
combined_df['State'] = combined_df.ORI_Code.str[0:2]
combined_df['State_number'] = combined_df['State'].map(state_to_FIPS)
combined_df['FIPS_Code'] = combined_df['State_number'] + combined_df['County_code']
combined_df['FIPS_Code'] = combined_df['FIPS_Code'].replace(fips_mapping)
combined_df.drop(['County_code','State','State_number'],axis=1, inplace=True)


In [8]:
combined_df.drop(['ORI_Code','Month','County','Suburban','MSA','State_Code','Division','Group','Agency_Name','State_Name'],axis=1,inplace=True)
#combined_df.groupby([]).mean()

In [9]:
combined_df

,Year,Offense_Code,Population_for_Agency,Adult-White,Adult-Black,Adult-Indian,Adult-Asian,Juvenile-White,Juvenile-Black,Juvenile-Indian,...,Female-25-29,Female-30-34,Female-35-39,Female-40-44,Female-45-49,Female-50-54,Female-55-59,Female-60-64,Female-Over-64,FIPS_Code
0,1,01A,151677.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001
1,1,02,151677.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001
2,1,03,151677.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,01001
3,1,04,151677.0,NaN,NaN,NaN,NaN,NaN,2.0,3.0,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,01001
4,1,05,151677.0,NaN,NaN,NaN,NaN,2.0,1.0,2.0,...,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412146,5,22,139573.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15005
1412147,5,24,139573.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,8.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,15005
1412148,5,26,139573.0,NaN,1.0,2.0,3.0,1.0,3.0,2.0,...,110.0,4.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,15005
1412149,5,28,139573.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,15005


In [10]:
population_df = combined_df.groupby(['FIPS_Code','Year']).agg({'Population_for_Agency':'mean'}).reset_index()

In [11]:
combined_df.drop(['Population_for_Agency'],axis=1,inplace=True)

In [12]:
combined_df = combined_df.groupby(['FIPS_Code','Year','Offense_Code']).sum()

In [13]:
combined_df.columns

Index(['Adult-White', 'Adult-Black', 'Adult-Indian', 'Adult-Asian',
       'Juvenile-White', 'Juvenile-Black', 'Juvenile-Indian', 'Juvenile-Asian',
       'Adult-Hispanic', 'Adult-Non-Hispanic', 'Juvenile-Hispanic',
       'Juvenile-Non-Hispanic', 'Male-Under-10', 'Male-10-12', 'Male-13-14',
       'Male-15', 'Male-16', 'Male-17', 'Male-18', 'Male-19', 'Male-20',
       'Male-21', 'Male-22', 'Male-23', 'Male-24', 'Male-25-29', 'Male-30-34',
       'Male-35-39', 'Male-40-44', 'Male-45-49', 'Male-50-54', 'Male-55-59',
       'Male-60-64', 'Male-Over-64', 'Female-Under-10', 'Female-10-12',
       'Female-13-14', 'Female-15', 'Female-16', 'Female-17', 'Female-18',
       'Female-19', 'Female-20', 'Female-21', 'Female-22', 'Female-23',
       'Female-24', 'Female-25-29', 'Female-30-34', 'Female-35-39',
       'Female-40-44', 'Female-45-49', 'Female-50-54', 'Female-55-59',
       'Female-60-64', 'Female-Over-64'],
      dtype='object')

In [14]:
combined_df['Arrests']= combined_df.iloc[:, 12:].sum(axis=1)

In [27]:
combined_df.head()

,FIPS_Code,Year,Offense_Code,Adult-White,Adult-Black,Adult-Indian,Adult-Asian,Juvenile-White,Juvenile-Black,Juvenile-Indian,...,Female-25-29,Female-30-34,Female-35-39,Female-40-44,Female-45-49,Female-50-54,Female-55-59,Female-60-64,Female-Over-64,Arrests
0,01001,1,01A,0.0,0.0,0.0,1.0,0.0,4.0,8.0,...,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,116.0
1,01001,1,01B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,01001,1,02,0.0,0.0,0.0,2.0,1.0,1.0,1.0,...,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0
3,01001,1,03,0.0,1.0,8.0,6.0,17.0,32.0,47.0,...,0.0,0.0,64.0,0.0,0.0,0.0,0.0,0.0,0.0,554.0
4,01001,1,04,0.0,3.0,9.0,12.0,11.0,11.0,25.0,...,2.0,8.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,1210.0


In [28]:
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Female-')))]
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Male-')))]
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Adult-')))]

In [29]:
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Juvenile-')))]

In [30]:
combined_df = combined_df.reset_index()

In [108]:
df = pd.merge(combined_df, population_df, on=["FIPS_Code", "Year"], how="left")

In [109]:
df['Arrest_rate'] = df["Arrests"].div(df["Population_for_Agency"].values)

In [110]:
df

,index,FIPS_Code,Year,Offense_Code,Arrests,Population_for_Agency,Arrest_rate
0,0,01001,1,01A,116.0,40591.141119,0.002858
1,1,01001,1,01B,6.0,40591.141119,0.000148
2,2,01001,1,02,55.0,40591.141119,0.001355
3,3,01001,1,03,554.0,40591.141119,0.013648
4,4,01001,1,04,1210.0,40591.141119,0.029809
...,...,...,...,...,...,...,...
411540,411540,56023,5,22,59.0,3063.568421,0.019259
411541,411541,56023,5,23,4.0,3063.568421,0.001306
411542,411542,56023,5,24,6.0,3063.568421,0.001959
411543,411543,56023,5,26,68.0,3063.568421,0.022196


In [111]:
df.drop(['Arrests','Population_for_Agency', 'index'],axis=1,inplace=True)

In [112]:
# df = df.groupby(['FIPS_Code','Year', 'Offense_Code'])['Arrest_rate'].apply(list).reset_index(name='Arrest_rate_by_offense_code')
# df = df.groupby(['FIPS_Code','Year', 'Offense_Code'])

In [113]:
df

,FIPS_Code,Year,Offense_Code,Arrest_rate
0,01001,1,01A,0.002858
1,01001,1,01B,0.000148
2,01001,1,02,0.001355
3,01001,1,03,0.013648
4,01001,1,04,0.029809
...,...,...,...,...
411540,56023,5,22,0.019259
411541,56023,5,23,0.001306
411542,56023,5,24,0.001959
411543,56023,5,26,0.022196


In [116]:
df = df.set_index(['FIPS_Code','Year','Offense_Code'])['Arrest_rate'].unstack().reset_index()

In [120]:
df

Offense_Code,FIPS_Code,Year,01A,01B,02,03,04,05,06,07,...,20,21,22,23,24,25,26,27,28,29
0,01001,1,0.002858,0.000148,0.001355,0.013648,0.029809,0.016654,0.118597,0.013525,...,0.000345,0.050996,0.015003,0.047646,0.008401,0.000739,0.243058,0.000000,0.000591,0.000887
1,01001,2,0.002509,0.000000,0.001599,0.013811,0.029242,0.014388,0.111551,0.011835,...,0.000333,0.049403,0.013922,0.047360,0.007616,0.000955,0.282785,0.000000,0.000466,0.000533
2,01001,3,0.002167,0.000000,0.001005,0.011840,0.031277,0.014544,0.109580,0.012198,...,0.000670,0.045128,0.010254,0.050534,0.007551,0.000313,0.297107,0.000000,0.000112,0.000559
3,01001,4,0.002291,0.000075,0.001743,0.013870,0.026994,0.019449,0.129989,0.012127,...,0.000548,0.039719,0.009861,0.041985,0.012401,0.000423,0.373159,0.000000,0.000075,0.000423
4,01001,5,0.002949,0.000057,0.001123,0.007934,0.024297,0.015697,0.080958,0.008733,...,0.000285,0.028483,0.006031,0.031679,0.009437,0.000476,0.217817,0.000000,0.000076,0.000400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15957,56023,1,0.000000,0.000000,0.000670,0.000000,0.001339,0.005692,0.003348,0.003348,...,0.001004,0.007701,0.010045,0.003013,0.001674,0.000000,0.030803,0.000000,0.000670,0.000670
15958,56023,2,0.000000,0.000000,0.000000,0.000000,0.000671,0.001676,0.002012,0.000000,...,0.000335,0.012741,0.006370,0.001006,0.000671,0.000000,0.022799,0.000671,0.000000,0.000000
15959,56023,3,0.000000,0.000000,0.000665,0.000000,0.000000,0.001994,0.001662,0.000332,...,0.000665,0.012297,0.006647,0.002659,0.001994,0.000000,0.029248,0.000000,0.002327,0.000000
15960,56023,4,0.000645,0.000000,0.000645,0.000000,0.000323,0.000323,0.002258,0.000000,...,0.000000,0.022258,0.010000,0.000645,0.002903,0.000323,0.025161,0.000000,0.000323,0.001935


In [119]:
df.fillna(0, inplace=True)

In [117]:
# df.groupby(['FIPS_Code','Year'])['Arrest_rate'].apply(dict).reset_index(name='Arrest_rate_by_offense_code')

KeyError: 'Column not found: Arrest_rate'

In [37]:
# df = df.groupby(['FIPS_Code'])['Arrest_rate_by_offense_code'].apply(list).reset_index(name='Arrest_rate_by_offense_code_by_year')

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [58]:
# df.head()

,FIPS_Code,Arrest_rate_by_offense_code_by_year
0,01001,"[[0.002857766419014756, 0.00014781550443179772..."
1,01002,"[[0.0006898938861658932, 0.0004734565885452208..."
2,01003,"[[0.0012091221379821734, 0.0001422496632920204..."
3,01004,"[[0.0003346205807101163, 0.0002868176406086712..."
4,01005,"[[0.0007680477021942328, 0.001152071553291349,..."


In [121]:
def get_all_counties():
    all_counties_set = set()
    for key, value in dictionary.items():
        all_counties_set.add(key)
        for v in value:
            all_counties_set.add(v)
    return list(all_counties_set) 

In [122]:
all_counties_list = get_all_counties()

In [123]:
all_counties_list

['01038',
 '06007',
 '06019',
 '01052',
 '01043',
 '06037',
 '32002',
 '01055',
 '01001',
 '06004',
 '01063',
 '02AST',
 '04010',
 '01044',
 '01047',
 '02001',
 '01003',
 '01057',
 '01041',
 '01061',
 '04013',
 '01004',
 '36030',
 '01002',
 '01025',
 '48SPD',
 '48HPD',
 '01005',
 '04003',
 '48101',
 '04002',
 '17CPD',
 '05060',
 '42PEP',
 '04011',
 '01059',
 '01019',
 '05072',
 '06001',
 '04007',
 '01011']

In [124]:
for i in range(len(all_counties_list)):
    if all_counties_list[i] in fips_mapping:
        all_counties_list[i] = fips_mapping.get(all_counties_list[i])

In [125]:
all_counties_list

['01038',
 '06007',
 '06019',
 '01052',
 '01043',
 '06037',
 '32002',
 '01055',
 '01001',
 '06004',
 '01063',
 '02AST',
 '04010',
 '01044',
 '01047',
 '02001',
 '01003',
 '01057',
 '01041',
 '01061',
 '04013',
 '01004',
 '36030',
 '01002',
 '01025',
 '48029',
 '48201',
 '01005',
 '04003',
 '48101',
 '04002',
 '17031',
 '05060',
 '42101',
 '04011',
 '01059',
 '01019',
 '05072',
 '06001',
 '04007',
 '01011']

In [73]:
# with open("all_counties_list_2016-2020.txt", "w") as file:
#     for county in all_counties_list:
#         file.write(county + "\n")

In [126]:
len(all_counties_list)

41

In [127]:
all_counties_df = df[df['FIPS_Code'].isin(all_counties_list)]

In [129]:
all_counties_df['FIPS_Code'] = all_counties_df['FIPS_Code'].astype(int)

C:\Users\dhruv\AppData\Local\Temp/ipykernel_3080/3327586077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_counties_df['FIPS_Code'] = all_counties_df['FIPS_Code'].astype(int)


In [130]:
all_counties_df

Offense_Code,FIPS_Code,Year,01A,01B,02,03,04,05,06,07,...,20,21,22,23,24,25,26,27,28,29
0,1001,1,0.002858,0.000148,0.001355,0.013648,0.029809,0.016654,0.118597,0.013525,...,0.000345,0.050996,0.015003,0.047646,0.008401,0.000739,0.243058,0.0,0.000591,0.000887
1,1001,2,0.002509,0.000000,0.001599,0.013811,0.029242,0.014388,0.111551,0.011835,...,0.000333,0.049403,0.013922,0.047360,0.007616,0.000955,0.282785,0.0,0.000466,0.000533
2,1001,3,0.002167,0.000000,0.001005,0.011840,0.031277,0.014544,0.109580,0.012198,...,0.000670,0.045128,0.010254,0.050534,0.007551,0.000313,0.297107,0.0,0.000112,0.000559
3,1001,4,0.002291,0.000075,0.001743,0.013870,0.026994,0.019449,0.129989,0.012127,...,0.000548,0.039719,0.009861,0.041985,0.012401,0.000423,0.373159,0.0,0.000075,0.000423
4,1001,5,0.002949,0.000057,0.001123,0.007934,0.024297,0.015697,0.080958,0.008733,...,0.000285,0.028483,0.006031,0.031679,0.009437,0.000476,0.217817,0.0,0.000076,0.000400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13939,48201,1,0.000169,0.000006,0.000236,0.001293,0.001974,0.001373,0.015104,0.000436,...,0.000035,0.009001,0.002243,0.028514,0.004980,0.000009,0.054138,0.0,0.000895,0.001298
13940,48201,2,0.000174,0.000016,0.000306,0.001405,0.001923,0.001390,0.015777,0.000324,...,0.000182,0.009228,0.002622,0.028467,0.005387,0.000007,0.060032,0.0,0.001865,0.000673
13941,48201,3,0.000174,0.000011,0.000445,0.001770,0.002402,0.001730,0.019026,0.000340,...,0.000037,0.012126,0.003128,0.031045,0.006428,0.000214,0.070854,0.0,0.001536,0.001250
13942,48201,4,0.000143,0.000018,0.000479,0.001713,0.002357,0.001841,0.018150,0.000370,...,0.000041,0.013331,0.002463,0.028477,0.007187,0.000391,0.066271,0.0,0.002262,0.000844


In [133]:
all_counties_df.to_csv("all_counties_df.csv", index=False)

In [87]:
### Read unemployment file
unemployment = pd.read_csv("Unemployment.csv")

In [88]:
unemployment

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,0,US,United States,NaN,NaN,NaN,"142,601,576","136,904,853","5,696,723",4.0,...,"163,140,305","157,154,185","5,986,120",3.7,"160,611,064","147,677,360","12,933,704",8.1,"65,712",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,147,173","2,047,731","99,442",4.6,...,"2,237,278","2,169,395","67,883",3.0,"2,230,118","2,099,062","131,056",5.9,"51,771",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,4.1,...,"26,281","25,561",720,2.7,"25,838","24,576","1,262",4.9,"58,233",112.5
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",3.7,...,"97,424","94,713","2,711",2.8,"96,763","91,338","5,425",5.6,"59,871",115.6
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,5.6,...,"8,502","8,177",325,3.8,"8,587","7,982",605,7.0,"35,972",69.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270,72145,PR,"Vega Baja Municipio, PR",1.0,1.0,1.0,"19,805","17,607","2,198",11.1,...,"13,172","11,910","1,262",9.6,NaN,NaN,NaN,NaN,NaN,NaN
3271,72147,PR,"Vieques Municipio, PR",7.0,12.0,0.0,"2,441","2,120",321,13.2,...,"2,616","2,433",183,7.0,NaN,NaN,NaN,NaN,NaN,NaN
3272,72149,PR,"Villalba Municipio, PR",2.0,2.0,1.0,"7,636","6,648",988,12.9,...,"7,492","6,307","1,185",15.8,NaN,NaN,NaN,NaN,NaN,NaN
3273,72151,PR,"Yabucoa Municipio, PR",1.0,1.0,1.0,"10,613","9,005","1,608",15.2,...,"8,840","7,685","1,155",13.1,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
joined_data = pd.merge(all_counties_df, unemployment, how="inner", on=["FIPS_Code"])

In [90]:
joined_data

,FIPS_Code,Arrest_rate_by_offense_code_by_year,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,1001,"[[0.002857766419014756, 0.00014781550443179772...",AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,...,"26,281","25,561",720,2.7,"25,838","24,576","1,262",4.9,"58,233",112.5
1,1003,"[[0.0012091221379821734, 0.0001422496632920204...",AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",...,"97,424","94,713","2,711",2.8,"96,763","91,338","5,425",5.6,"59,871",115.6
2,1005,"[[0.0007680477021942328, 0.001152071553291349,...",AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,...,"8,502","8,177",325,3.8,"8,587","7,982",605,7.0,"35,972",69.5
3,1011,"[[0.000525640253721772, 0.0014717927104209614,...",AL,"Bullock County, AL",6.0,6.0,0.0,"3,997","3,656",341,...,"4,785","4,608",177,3.7,"4,818","4,553",265,5.5,"31,906",61.6
4,1019,"[[0.0005364379980491538, 0.0004693832482930096...",AL,"Cherokee County, AL",6.0,6.0,0.0,"11,378","10,862",516,...,"11,726","11,386",340,2.9,"11,513","10,983",530,4.6,"45,982",88.8
5,1025,"[[0.00026339616429335745, 0.000131698082146678...",AL,"Clarke County, AL",7.0,11.0,0.0,"11,677","10,814",863,...,"7,588","7,151",437,5.8,"7,769","7,069",700,9.0,"43,822",84.6
6,1041,"[[0.00029179573805853645, 6.484349734634143e-0...",AL,"Crenshaw County, AL",8.0,6.0,0.0,"6,155","5,800",355,...,"6,436","6,253",183,2.8,"6,253","5,896",357,5.7,"43,309",83.7
7,1043,"[[0.0003452329621239883, 0.0002589247215929912...",AL,"Cullman County, AL",4.0,3.0,0.0,"38,898","37,274","1,624",...,"38,541","37,533","1,008",2.6,"38,310","36,660","1,650",4.3,"50,897",98.3
8,1047,"[[0.00034601950865978734, 9.436895690721472e-0...",AL,"Dallas County, AL",4.0,5.0,0.0,"17,948","16,487","1,461",...,"14,655","13,850",805,5.5,"14,619","13,049","1,570",10.7,"34,034",65.7
9,1055,"[[0.0007929314336017737, 0.0008920478628019954...",AL,"Etowah County, AL",3.0,2.0,1.0,"48,496","45,652","2,844",...,"42,462","41,026","1,436",3.4,"40,724","37,598","3,126",7.7,"43,047",83.1


In [91]:
joined_data.to_csv("unemployment_data_joined_2001-2005.csv")

In [ ]:
unemployment['FIPS_Code'] = unemployment['FIPS_Code'].astype(str)
unemployment[unemployment['FIPS_Code'].isin([x.lstrip('0') for x in all_counties_list])]

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,4.1,...,"26,281","25,561",720,2.7,"25,838","24,576","1,262",4.9,"58,233",112.5
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",3.7,...,"97,424","94,713","2,711",2.8,"96,763","91,338","5,425",5.6,"59,871",115.6
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,5.6,...,"8,502","8,177",325,3.8,"8,587","7,982",605,7.0,"35,972",69.5
7,1011,AL,"Bullock County, AL",6.0,6.0,0.0,"3,997","3,656",341,8.5,...,"4,785","4,608",177,3.7,"4,818","4,553",265,5.5,"31,906",61.6
11,1019,AL,"Cherokee County, AL",6.0,6.0,0.0,"11,378","10,862",516,4.5,...,"11,726","11,386",340,2.9,"11,513","10,983",530,4.6,"45,982",88.8
14,1025,AL,"Clarke County, AL",7.0,11.0,0.0,"11,677","10,814",863,7.4,...,"7,588","7,151",437,5.8,"7,769","7,069",700,9.0,"43,822",84.6
22,1041,AL,"Crenshaw County, AL",8.0,6.0,0.0,"6,155","5,800",355,5.8,...,"6,436","6,253",183,2.8,"6,253","5,896",357,5.7,"43,309",83.7
23,1043,AL,"Cullman County, AL",4.0,3.0,0.0,"38,898","37,274","1,624",4.2,...,"38,541","37,533","1,008",2.6,"38,310","36,660","1,650",4.3,"50,897",98.3
25,1047,AL,"Dallas County, AL",4.0,5.0,0.0,"17,948","16,487","1,461",8.1,...,"14,655","13,850",805,5.5,"14,619","13,049","1,570",10.7,"34,034",65.7
29,1055,AL,"Etowah County, AL",3.0,2.0,1.0,"48,496","45,652","2,844",5.9,...,"42,462","41,026","1,436",3.4,"40,724","37,598","3,126",7.7,"43,047",83.1


In [ ]:
set([x.lstrip('0') for x in all_counties_list]) - set(unemployment['FIPS_Code'])

{'1002',
 '1004',
 '1038',
 '1044',
 '1052',
 '17CPD',
 '2001',
 '2AST',
 '32002',
 '36030',
 '4002',
 '4010',
 '42PEP',
 '48HPD',
 '48SPD',
 '5060',
 '5072',
 '6004'}

In [ ]:
unemployment['FIPS_Code'].unique()[0:10]

array(['0', '1000', '1001', '1003', '1005', '1007', '1009', '1011',
       '1013', '1015'], dtype=object)